                                               Funciones para la API:

In [36]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
#Se cargan los datos
DF_GAMES = pd.read_parquet('../Parquets/ETL_Steam_Games.gzip')
DF_REVIEWS = pd.read_parquet('../Parquets/Review_Sentiment_Analysis.gzip')
DF_ITEMS = pd.read_parquet('../Parquets/ETL_user_items.gzip')


                                            FUNCION 1

In [37]:
def PlayTimeGenre(genero: str):

    #Se une lo datos relevantes
    merged_df = pd.merge(DF_GAMES, DF_ITEMS, on='Item_Id')

     #Se filtra por género
    genre_df = merged_df[merged_df['Genres'] == genero]

    #Se agrupa y sumar horas jugadas
    grouped_df = genre_df.groupby('Release_Year')['Playtime_Forever'].sum().reset_index()

    #Se busca el año con más horas jugadas
    max_year = grouped_df.loc[grouped_df['Playtime_Forever'].idxmax(), 'Release_Year']
    result = {"Año de lanzamiento con más horas jugadas para Género " + genero: int(max_year)}
    return result

In [38]:
PLAYTIME = PlayTimeGenre('Strategy')
PLAYTIME

{'Año de lanzamiento con más horas jugadas para Género Strategy': 2013}

                                            FUNCION 2

In [39]:
def UserForGenre(genero: str):
    #Se une lo datos relevantes
    merged_df = pd.merge(DF_GAMES, DF_ITEMS, on='Item_Id')

    #Se filtra por género
    genre_df = merged_df[merged_df['Genres'] == genero]

    #Se agrupa por usuario y año, y sumar horas jugadas
    grouped_df = genre_df.groupby(['User_Id', 'Release_Year'])['Playtime_Forever'].sum().reset_index()

    #Se busca el usuario con más horas jugadas
    max_user_row = grouped_df.loc[grouped_df['Playtime_Forever'].idxmax()]
    max_user = max_user_row['User_Id']

    #Se crea la lista de acumulación de horas jugadas por año sin duplicados
    unique_years = grouped_df['Release_Year'].unique()
    hours_list = [{"Año": int(year), "Horas": int(grouped_df[grouped_df['Release_Year'] == year]['Playtime_Forever'].sum())} for year in unique_years]

    #Se ordena la lista por año
    hours_list = sorted(hours_list, key=lambda x: x['Año'])
    result = {"Usuario con más horas jugadas para Género " + genero: max_user,
              "Horas jugadas": hours_list}
    return result


In [40]:
GENRE_HOURS =UserForGenre('Casual')
GENRE_HOURS

{'Usuario con más horas jugadas para Género Casual': 'GeometryWarsFTW',
 'Horas jugadas': [{'Año': 1995, 'Horas': 385},
  {'Año': 1999, 'Horas': 222},
  {'Año': 2001, 'Horas': 675},
  {'Año': 2005, 'Horas': 6707},
  {'Año': 2007, 'Horas': 258750},
  {'Año': 2008, 'Horas': 15278},
  {'Año': 2009, 'Horas': 6955},
  {'Año': 2010, 'Horas': 74735},
  {'Año': 2011, 'Horas': 15066},
  {'Año': 2012, 'Horas': 1701},
  {'Año': 2013, 'Horas': 57625},
  {'Año': 2014, 'Horas': 16471},
  {'Año': 2015, 'Horas': 36796},
  {'Año': 2016, 'Horas': 25373}]}

                                                FUNCION 3

In [41]:
def UsersRecommend(año: int):
    '''Recordar que como se vió en el EDA, esta funcion solo admite datos de año entre2010 y 2015'''
    #Se une datos relevantes
    merged_df = pd.merge(DF_GAMES, DF_REVIEWS, on='Item_Id')

    #Se filtra por año y recomendaciones positivas/neutrales
    year_df = merged_df[merged_df['Year_Posted'] == año]
    positive_neutral_df = year_df[year_df['Sentiment_Analysis'].isin([1, 2]) & year_df['Recommend']]

    #Se cuentan las recomendaciones por juego
    top_games = positive_neutral_df.groupby('App_Name')['Recommend'].sum().reset_index()

    #Se odena y obtener el top 3
    top_games = top_games.sort_values(by='Recommend', ascending=False).head(3)

    #Se crea la lista de salida
    result_list = [{"Puesto {}: ".format(i+1): game} for i, game in enumerate(top_games['App_Name'])]
    return result_list

In [42]:
MOST_RECO = UsersRecommend(2010)
MOST_RECO

[{'Puesto 1: ': 'Team Fortress 2'},
 {'Puesto 2: ': 'Counter-Strike: Source'},
 {'Puesto 3: ': 'Portal'}]

                                                FUNCION 4

In [43]:
def UsersWorstDeveloper(año: int):
    #Se filtra juegos lanzados en el año dado
    games_year_df = DF_GAMES[DF_GAMES["Release_Year"] == año]

    #Se ue los DataFrames para obtener la información necesaria
    merged_df = pd.merge(DF_REVIEWS, games_year_df, on="Item_Id")
    final_df = pd.merge(merged_df, DF_ITEMS, on="Item_Id")

    # Filtrar juegos con recomendaciones negativas
    worst_games_df = final_df[(final_df["Recommend"] == False) & (final_df["Sentiment_Analysis"] == 0)]

    #Se agrupa por desarrollador y contar el número de juegos con recomendaciones negativas
    developer_counts = worst_games_df.groupby("Developer")["Item_Id"].count().reset_index()
    
    #TOp 3 peores
    top_developers = developer_counts.nlargest(3, "Item_Id")
    result = [{"Puesto {}: ".format(i): {"Developer": developer_row.Developer}} for i, developer_row in enumerate(top_developers.itertuples(index=False), start=1)]

    return result

In [44]:
LEAST_RECO = UsersWorstDeveloper(2012)
LEAST_RECO

[{'Puesto 1: ': {'Developer': 'Jagex Limited'}},
 {'Puesto 2: ': {'Developer': 'Hardsuit Labs'}},
 {'Puesto 3: ': {'Developer': 'Ubisoft Montreal Massive Entertainment And Ubisoft Shanghai'}}]

                                              FUNCION 5

In [45]:
def sentiment_analysis(desarrolladora: str):
    #Se fitran los datos para la empresa dada
    games_df_filtered = DF_GAMES[DF_GAMES['Developer'] == desarrolladora]
    reviews_df_filtered = pd.merge(DF_REVIEWS, games_df_filtered[['Item_Id']], on='Item_Id')
    
    #Se cuenta la cantidad de registros de reseñas por categoría de sentimiento
    sentiment_counts = reviews_df_filtered['Sentiment_Analysis'].value_counts().to_dict()
    
    #Se crea el diccionario de salida
    output_dict = {desarrolladora: {'Negative': sentiment_counts.get(0, 0),
                                    'Neutral': sentiment_counts.get(1, 0),
                                    'Positive': sentiment_counts.get(2, 0)}}
    return output_dict

In [46]:
SENTIMENT_DEVS = sentiment_analysis('Valve')
SENTIMENT_DEVS

{'Valve': {'Negative': 337, 'Neutral': 985, 'Positive': 2629}}

                                    SISTEMA RECOMENDACION  ITEM-ITEM

La función recomendacion_juego, carga datos de Recomendacion.parquet, archivo creado en el EDA del proyecto. Posteriormente, se crea una matriz donde las filas son los usuarios, las columnas son los juegos y los valores son las calificaciones (Rating). Se rellenan lo valores faltantes con 3, esto debido a qué, como se observa en el EDA, la gran mayoría de calificaciones . Luego, normaliza estas calificaciones utilizando MinMaxScaler para ponerlas en el rango [0, 1]. Después, calcula la similitud de coseno entre juegos, encuentra el índice del juego dado en la matriz de similitud, identifica los cinco juegos más similares (excluyendo el propio juego) y devuelve sus nombres como recomendaciones. Este enfoque ayuda a proporcionar recomendaciones de juegos más robustas al tener en cuenta las similitudes relativas entre los juegos y normalizar las calificaciones para evitar sesgos debidos a diferentes escalas.

In [59]:
recomendaciones = pd.read_parquet('../Parquets/Recomendacion.gzip')
def recomendacion_juego(Item_Name: str):
 #Se crea la matriz de calificaiciones:
 matrix = recomendaciones.pivot(index='User_Id', columns='Item_Name', values='Rating').fillna(1)

 #Se normaliza las calificaciones para que esten en el rango correcto:
 scaler = MinMaxScaler()
 matrix_scaled = scaler.fit_transform(matrix)

 #Se calcula la similitud del coseno entre los juegos
 similarity_matrix = cosine_similarity(matrix_scaled.T)

 #Se busca el índice del juego dado en la matriz de similitud
 index = matrix.columns.get_loc(Item_Name)

 #Se buscan los índices de los juegos similares
 similar_indices = similarity_matrix[index].argsort()[-6:-1]
 recommended_games = matrix.columns[similar_indices]

 return f"Porque jugaste '{Item_Name}', te pueden interesar:{', '.join(recommended_games)}"


In [60]:
recomendacion = recomendacion_juego('Killing Floor')
recomendacion

"Porque jugaste 'Killing Floor', te pueden interesar:Lucius II, Wrack, Nexuiz, Deus Ex: Revision, Brothers in Arms: Hell's Highway"